###Number Mind

In [32]:
!pip install python-sat
from pysat.solvers import Minisat22
from random import seed
import random
from random import randint
import itertools
import numpy as np
from time import process_time


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

def digit(n,r):
    return n*numdigit + r + 1

def num2CNFcode(num,numdigit):

  CNFcode = [0] * numdigit
  for x in range(numdigit):
    CNFcode[numdigit - x -1]= digit(int(num%10), numdigit - x -1)
    num=int(num/10)
  return CNFcode

def decodify(coded_vector):
  decoded_vector = [0] * len(coded_vector)
  for number in coded_vector:
    for pos in range(len(coded_vector)):
      if (number - 1 - pos)%len(coded_vector) == 0:
        decoded_vector[pos] = (number -1 - pos) / len(coded_vector)
  decoded_vector = ''.join(str(int(i)) for i in decoded_vector)
  return decoded_vector

In [109]:
numdigit = 17
#initial guess

t_start = process_time() 

secret_code = random_with_N_digits(numdigit)
print ("The secret code is :",secret_code)  
code_c = num2CNFcode(secret_code,numdigit)
print ("The secret code is :",code_c)  


initial_guess = int (''.join(str(int(i)) for i in [8] *numdigit))
print ("initial guess is :",initial_guess)  
guess_c = num2CNFcode(initial_guess,numdigit)

clauses=[]
num = Minisat22()
#each position should have at least a number
for r in range(numdigit):
  num.add_clause([digit(n,r) for n in range(10)])
#A position can not have two value
for r in range(numdigit):
  for n in range(10):
    for n1 in range(10): 
      if n!=n1:
        num.add_clause([-digit(n,r),-digit(n1,r)])


STOP = False

while STOP==False :
  # find the number of correct digit
  num_correct= 0
  for i in guess_c:
    if i in code_c:
      num_correct = num_correct+1
  if num_correct==numdigit:
    break
  print("Method's guess is : {}, {} correct".format(decodify(guess_c),num_correct))
  new_guess = []
  if num_correct == 0:
    for x in guess_c:
      num.add_clause([-x])
  elif num_correct == len(guess_c):
    STOP=True
  else:
    cnf_least = list(itertools.combinations(guess_c,len(guess_c) - int(num_correct) + 1))
    guess_neg = [ -x for x in guess_c]
    cnf_most = list(itertools.combinations(guess_neg, 1 + int(num_correct)))
    for x,y in zip(cnf_least, cnf_most):
      num.add_clause(list(x))
      num.add_clause(list(y))

  result_flag= num.solve()
  s=num
  for m in s.enum_models():
    m_positive = [x for x in m if x>0 ]
    if len(m_positive) >= len(guess_c):
      new_guess= m_positive[:len(guess_c)]
      break
  guess_c = new_guess

print("=======================================")
res = decodify(guess_c)
print ("The secret code is :{}".format(secret_code ))
print ("The Final guess is :{}".format(res ))

print("=======================================")
t_stop = process_time()
time_GD = t_stop - t_start
print("CPU time:", time_GD)


The secret code is : 40410379117803749
The secret code is : [69, 2, 71, 21, 5, 57, 126, 161, 26, 27, 130, 148, 13, 65, 134, 101, 170]
initial guess is : 88888888888888888
Method's guess is : 88888888888888898, 1 correct
Method's guess is : 80000000000000000, 3 correct
Method's guess is : 80011111111111111, 4 correct
Method's guess is : 70011822222222220, 2 correct
Method's guess is : 89011822222222220, 1 correct
Method's guess is : 89011822222222207, 1 correct
Method's guess is : 89011822222222077, 1 correct
Method's guess is : 89011022222222777, 2 correct
Method's guess is : 80911022222222077, 2 correct
Method's guess is : 90099057119845556, 5 correct
Method's guess is : 60047098118496963, 3 correct
Method's guess is : 80073959819367834, 3 correct
Method's guess is : 90058465617740717, 4 correct
Method's guess is : 90069784915071675, 2 correct
Method's guess is : 10074176146980557, 4 correct
Method's guess is : 20098940153535197, 2 correct
Method's guess is : 30036066771144858, 2 corr